<a href="https://colab.research.google.com/github/Aradhya6394/Artificial-Intelligence-2025/blob/main/Min_Max_Alphabeta_pruning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
"""
This program is an implementation of the **Tic-Tac-Toe game**,
where the human player (X) competes against an **AI opponent (O)**.
The AI uses the **Minimax algorithm** with **Alpha-Beta Pruning**
to make optimal decisions — meaning the computer never loses
if it plays correctly.

Key Concepts:
1. **Tic Tac Toe** – A 3x3 grid game where players alternate turns
   placing X or O, trying to get 3 in a row (horizontal, vertical, diagonal).

2. **Minimax Algorithm** – A decision-making algorithm used in games.
   - Maximizing player (X): tries to get the maximum score.
   - Minimizing player (O): tries to minimize the score.

3. **Alpha-Beta Pruning** – Optimization technique for Minimax.
   It skips (prunes) branches that cannot affect the final decision,
   reducing the number of states the AI evaluates → making it faster.

Features:
- Human (X) vs Computer (O)
- Computer uses Minimax + Alpha-Beta Pruning
- Displays number of states the AI evaluated
- Detects win, loss, or draw automatically
------------------------------------------------------------
"""

class TicTacToe:
    def __init__(self):
        # Initialize empty 3x3 board as a list of 9 spaces
        self.board = [" " for _ in range(9)]
        self.call_count = 0   # Count how many states AI evaluated (for display)

    # ---------------- Display Functions ---------------- #
    def print_board(self):
        """Print the current board in a 3x3 grid format."""
        print(self.board[0]+"|"+self.board[1]+"|"+self.board[2])
        print("-+-+-")
        print(self.board[3]+"|"+self.board[4]+"|"+self.board[5])
        print("-+-+-")
        print(self.board[6]+"|"+self.board[7]+"|"+self.board[8])

    # ---------------- Basic Game Logic ---------------- #
    def player(self, state=None):
        """Return which player's turn it is (X or O)."""
        s = state if state else self.board
        X_count = s.count("X")
        O_count = s.count("O")
        # X always starts first; if equal count → X's turn
        return "X" if X_count == O_count else "O"

    def actions(self, state=None):
        """Return list of available (empty) positions for the next move."""
        s = state if state else self.board
        return [i for i in range(9) if s[i] == " "]

    def result(self, state, action):
        """Return new board state after applying a move."""
        new_state = state.copy()
        new_state[action] = self.player(state)  # Place the symbol (X or O)
        return new_state

    def check_winner(self, player, state=None):
        """Check if the given player (X/O) has won."""
        s = state if state else self.board
        # All winning combinations (rows, columns, diagonals)
        win_conditions = [
            [0,1,2],[3,4,5],[6,7,8], # rows
            [0,3,6],[1,4,7],[2,5,8], # columns
            [0,4,8],[2,4,6]          # diagonals
        ]
        # Return True if any winning condition is met
        for cond in win_conditions:
            if s[cond[0]] == s[cond[1]] == s[cond[2]] == player:
                return True
        return False

    def terminal(self, state=None):
        """Return True if the game has ended (win or draw)."""
        s = state if state else self.board
        # Game ends if X or O wins or if board is full
        return (self.check_winner("X", s) or
                self.check_winner("O", s) or
                " " not in s)

    def utility(self, state=None):
        """
        Return the game score:
        +1 if X wins,
        -1 if O wins,
         0 if draw or ongoing.
        """
        s = state if state else self.board
        if self.check_winner("X", s): return 1
        elif self.check_winner("O", s): return -1
        else: return 0

    # ---------------- Minimax Algorithm with Alpha-Beta ---------------- #
    def max_value(self, state, alpha, beta):
        """Return the maximum score achievable by X (the maximizing player)."""
        self.call_count += 1
        if self.terminal(state):
            return self.utility(state)

        v = float("-inf")
        for action in self.actions(state):
            v = max(v, self.min_value(self.result(state, action), alpha, beta))
            # Beta cutoff: stop if value is already worse for minimizing player
            if v >= beta:
                return v
            alpha = max(alpha, v)
        return v

    def min_value(self, state, alpha, beta):
        """Return the minimum score achievable by O (the minimizing player)."""
        self.call_count += 1
        if self.terminal(state):
            return self.utility(state)

        v = float("inf")
        for action in self.actions(state):
            v = min(v, self.max_value(self.result(state, action), alpha, beta))
            # Alpha cutoff: stop if value is already worse for maximizing player
            if v <= alpha:
                return v
            beta = min(beta, v)
        return v

    def minimax_decision(self, state=None):
        """Return the best move for the current player using Minimax + Alpha-Beta."""
        self.call_count = 0
        s = state if state else self.board
        player = self.player(s)

        if player == "X":   # X → Maximizing player
            best_val = float("-inf")
            best_action = None
            for action in self.actions(s):
                val = self.min_value(self.result(s, action), float("-inf"), float("inf"))
                if val > best_val:
                    best_val = val
                    best_action = action
        else:               # O → Minimizing player
            best_val = float("inf")
            best_action = None
            for action in self.actions(s):
                val = self.max_value(self.result(s, action), float("-inf"), float("inf"))
                if val < best_val:
                    best_val = val
                    best_action = action

        print(f"AI evaluated {self.call_count} states before choosing move {best_action+1}")
        return best_action

    # ---------------- Game Loop ---------------- #
    def play(self):
        """Start the game: Human (X) vs AI (O)."""
        state = self.board
        print("Welcome to Tic Tac Toe! (You are X, AI is O)\n")
        print("Board positions are numbered like this:")
        print("1|2|3")
        print("-+-+-")
        print("4|5|6")
        print("-+-+-")
        print("7|8|9\n")

        # Continue until game reaches a terminal state (win or draw)
        while not self.terminal(state):
            self.print_board()

            if self.player(state) == "X":   # Human's turn
                try:
                    move = int(input("Enter your move (1-9): ")) - 1
                    if move not in self.actions(state):
                        print("Invalid move! Try again.")
                        continue
                except ValueError:
                    print("Please enter a number between 1 and 9.")
                    continue
            else:                           # AI's turn
                print("AI is thinking...")
                move = self.minimax_decision(state)

            # Apply chosen move
            state = self.result(state, move)
            self.board = state  # Update main board

        # Display final board and result
        self.print_board()
        if self.utility(state) == 1:
            print("🎉 You (X) win! Congratulations!")
        elif self.utility(state) == -1:
            print("🤖 AI (O) wins! Better luck next time.")
        else:
            print("🤝 It's a draw!")

# ---------------- Run the Game ---------------- #
if __name__ == "__main__":
    game = TicTacToe()
    game.play()


Welcome to Tic Tac Toe! (You are X, AI is O)

Board positions are numbered like this:
1|2|3
-+-+-
4|5|6
-+-+-
7|8|9

 | | 
-+-+-
 | | 
-+-+-
 | | 
Enter your move (1-9): 9
 | | 
-+-+-
 | | 
-+-+-
 | |X
AI is thinking...
AI evaluated 6200 states before choosing move 5
 | | 
-+-+-
 |O| 
-+-+-
 | |X
Enter your move (1-9): 3
 | |X
-+-+-
 |O| 
-+-+-
 | |X
AI is thinking...
AI evaluated 368 states before choosing move 6
 | |X
-+-+-
 |O|O
-+-+-
 | |X
Enter your move (1-9): 4
 | |X
-+-+-
X|O|O
-+-+-
 | |X
AI is thinking...
AI evaluated 55 states before choosing move 1
O| |X
-+-+-
X|O|O
-+-+-
 | |X
Enter your move (1-9): 8
O| |X
-+-+-
X|O|O
-+-+-
 |X|X
AI is thinking...
AI evaluated 4 states before choosing move 7
O| |X
-+-+-
X|O|O
-+-+-
O|X|X
Enter your move (1-9): 2
O|X|X
-+-+-
X|O|O
-+-+-
O|X|X
🤝 It's a draw!
